<a href="https://colab.research.google.com/github/DomizianoScarcelli/big-data-project/blob/nn-model/user_based_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuration

Here we configure the environment. Since I alternated from Google Colab to Local development, I define a LOCAL variable that allows me to know in which environment I am. 

In [1]:
import os
def is_running_on_colab():
    return "COLAB_GPU" in os.environ

LOCAL = not is_running_on_colab()

In [2]:
#@title Download necessary libraries
if not LOCAL:
    !pip install pyspark -qq
    !pip install -U -q PyDrive -qq
    !apt install openjdk-8-jdk-headless -qq

In [3]:
#@title Imports
import numpy as np
import json
import time
import math
import sys


import pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, FloatType, LongType
from pyspark import SparkConf
from pyspark.ml.linalg import SparseVector, VectorUDT

from tqdm.notebook import tqdm

if not LOCAL:
    from google.colab import drive

from typing import Tuple, Callable, List
from functools import reduce

In [4]:
#@title Set up variables
if not LOCAL:
    JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
    GDRIVE_DIR = "/content/drive"
    GDRIVE_HOME_DIR = GDRIVE_DIR + "/MyDrive"
    GDRIVE_DATA_DIR = GDRIVE_HOME_DIR + "/Big Data/datasets"
    DATASET_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_friendly_spotify_playlist_dataset")
    AUDIO_FEATURES_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_track_features")
    LITTLE_SLICE_FILE = os.path.join(GDRIVE_DATA_DIR, "little_slice")
    SMALL_SLICE_FLIE = os.path.join(GDRIVE_DATA_DIR, "small_slice")
    LITTLE_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "little_slice_audio_features")
    MICRO_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "micro_slice_audio_features")
    SPLITTED_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "splitted_pyspark_track_features")
    SAVED_DFS_PATH = os.path.join(GDRIVE_DATA_DIR, "saved_dfs")
    SAVED_MODELS = os.path.join(GDRIVE_DATA_DIR, "saved_models")
else:
    GDRIVE_DATA_DIR = os.path.abspath("./data")
    SAVED_DFS_PATH = os.path.join(GDRIVE_DATA_DIR, "saved_dfs")
    SAVED_MODELS = os.path.join(GDRIVE_DATA_DIR, "saved_models")
    DATASET_FILE = os.path.join(GDRIVE_DATA_DIR, "full_dataset")
    SMALL_SLICE_FLIE = os.path.join(GDRIVE_DATA_DIR, "small_slice")
    JAVA_HOME = "/opt/homebrew/opt/openjdk"
RANDOM_SEED = 42 # for reproducibility
os.environ["JAVA_HOME"] = JAVA_HOME
os.environ["PYSPARK_PYTHON"]="python"

In [5]:
#@title Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '100G').\
                set("spark.executor.extraJavaOptions", "-XX:+UseG1GC").\
                setAppName("PySparkTutorial").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

23/06/29 18:09:28 WARN Utils: Your hostname, MacBook-Air-di-Domiziano.local resolves to a loopback address: 127.0.0.1; using 192.168.1.175 instead (on interface en0)
23/06/29 18:09:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/29 18:09:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/29 18:09:28 WARN Utils: Service 'SparkUI' could not bind on port 4050. Attempting port 4051.


In [6]:
if not LOCAL:
    drive.mount(GDRIVE_DIR, force_remount=True)

In [7]:
#@title Check if everything is ok
spark, sc._conf.getAll()

(<pyspark.sql.session.SparkSession at 0x10f660070>,
 [('spark.app.submitTime', '1688054968219'),
  ('spark.executor.extraJavaOptions',
   '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false -XX:+UseG1GC'),
  ('spark.app.startTime', '1688054968300

# Load DataFrame

Define the `DataFrame` schemas and load the primary `DataFrame`

In [8]:
song_schema = StructType([
    StructField("pos", IntegerType(), True),
    StructField("artist_name", StringType(), True),
    StructField("track_uri", StringType(), True),
    StructField("artist_uri", StringType(), True),
    StructField("track_name", StringType(), True),
    StructField("album_uri", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("album_name", StringType(), True)
])

playlist_schema = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", IntegerType(), True),
    StructField("modified_at", IntegerType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", ArrayType(song_schema), True),
    StructField("num_edits", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("num_artists", IntegerType(), True),
])

playlist_schema_mapped = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", IntegerType(), True),
    StructField("modified_at", IntegerType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", VectorUDT(), True),
    StructField("num_edits", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("num_artists", IntegerType(), True),
])

In [9]:
slice_df = spark.read.schema(playlist_schema).json(SMALL_SLICE_FLIE, multiLine=True)

# User-Based Collaborative Filtering
Note: The users are the playlists, the items are the songs and the ratings are 0 if the song is not in the playlist, 1 otherwise.

We have to define a function $sim(u,v)$ that defines the similarity between two users based on their ratings.

We represent the ratings $r_u \in \mathbb{R}^n$ as the $n$ dimensional vector that represents the ratings of the user $u$, where $n$ is the number of total songs in the dataset.

As the similarity function we can use Jaccard similarity.
\begin{equation}
sim(u,v) = J(r_u, r_v) = \frac{|r_u \cap r_v|}{|r_u \cup r_v|}
\end{equation}

Jaccard similarity ignores rating values, but we don't care here since the ratings are binary. In case of discrete value ratings we can use cosine similarity, or better pearson's correlation.

Done that, and defined as ${U^k}$ the neighborhood of $u$ ($k$ most similar users to $u$), we define the set of items rated by $u$'s neighborhood as

\begin{equation}
I^k = \{i \in I : \mathbf{r_{u,i}} \downarrow \land u \in U^k\}
\end{equation}

The rating for the item $i$ to the user $u$ will just be $\mathbf{r_u[i]}$.

In [10]:
DEBUG = False #If True, execute code that helps to debug the code

Let's load the song embedding encodings, and the `DataFrame` that maps each song to a position

In [11]:
NUM_PLAYLISTS = 100_000
SONGS_EMBEDDINGS_PATH = os.path.join(SAVED_DFS_PATH, f"songs_embeddings-{NUM_PLAYLISTS}.json")
SONGS_INFO_DF = os.path.join(SAVED_DFS_PATH, f"songs_info_df-{NUM_PLAYLISTS}.json") #TODO: Little bug this is songs_df, meaning it hasn't got any info, but we don't actually care.
RATING_VECTOR_LENGTH_PATH = os.path.join(SAVED_DFS_PATH, f"songs_vector_length-{NUM_PLAYLISTS}.txt")
with open(RATING_VECTOR_LENGTH_PATH, "r") as f:
  RATING_VECTOR_LENGTH = int(f.read())

songs_embeddings = spark.read.schema(playlist_schema_mapped).json(SONGS_EMBEDDINGS_PATH)
song_pos_mapping = spark.read.json(SONGS_INFO_DF)

In [12]:
songs_embeddings.show()

+-------------+-------------+----+-----------+----------+----------+-------------+--------------------+---------+-----------+-----------+
|         name|collaborative| pid|modified_at|num_tracks|num_albums|num_followers|              tracks|num_edits|duration_ms|num_artists|
+-------------+-------------+----+-----------+----------+----------+-------------+--------------------+---------+-----------+-----------+
|       disney|        false|1000| 1457827200|       189|        16|            1|(681805,[126,1903...|        4|   31428282|         65|
|Indie Electro|        false|1001| 1417824000|       165|        18|            2|(681805,[17322,18...|        2|   38241566|          8|
|  jack & jack|        false|1002| 1465430400|        17|        14|            1|(681805,[4591,952...|        3|    3549358|          3|
|        vibes|        false|1003| 1498435200|       225|       195|            2|(681805,[392,431,...|       91|   51242585|        157|
|        Indie|        false|1004|

In [13]:
song_pos_mapping.show()

+---+--------------------+
|pos|           track_uri|
+---+--------------------+
|  0|spotify:track:1mr...|
|  1|spotify:track:1Uv...|
|  2|spotify:track:4WR...|
|  3|spotify:track:7B6...|
|  4|spotify:track:2Gy...|
|  5|spotify:track:7AO...|
|  6|spotify:track:48Z...|
|  7|spotify:track:1Um...|
|  8|spotify:track:7MO...|
|  9|spotify:track:27P...|
| 10|spotify:track:6lt...|
| 11|spotify:track:1yz...|
| 12|spotify:track:5Mz...|
| 13|spotify:track:3BU...|
| 14|spotify:track:4Cl...|
| 15|spotify:track:2dN...|
| 16|spotify:track:341...|
| 17|spotify:track:7ja...|
| 18|spotify:track:4eQ...|
| 19|spotify:track:6fy...|
+---+--------------------+
only showing top 20 rows



Preprocessing the dataframe in order to associate to each `track_uri` an integer, that will represent the position of the track in the `rating_vector`. This is the same function that generates the `songs_embeddings`, but I also need it here because I need to convert the input playlist to continuate when doing performance evaluation.

In [14]:
track_uri_to_id = song_pos_mapping.select('track_uri', 'pos').rdd.collectAsMap()
def map_track_df_to_pos(playlist_df: DataFrame) -> DataFrame:
    """
    Returns a DataFrames containing the playlists, but the tracks are represented as a binary sparse vector.
    """

    @F.udf(returnType=VectorUDT())
    def extract_vector(tracks):
      pos_list = set()

      def reduce_fn(pos_list, row):
          pos_list.add(track_uri_to_id.get(row.track_uri))
          return pos_list
      
      pos_list = reduce(reduce_fn, tracks, pos_list)
      
      return SparseVector(RATING_VECTOR_LENGTH, sorted(list(pos_list)), [1 for _ in pos_list])

    # Apply the mapping UDF on the "tracks" column of the slice_df dataframe
    mapped_df = playlist_df.withColumn('tracks', extract_vector(F.col('tracks')))

    return mapped_df

I could transform the `song_pos_mapping` into a python dictionary, since it requires very little memory (about 20 MB).

In [15]:
print("The size of the track_uri -> position mapping dictionary is {} MB".format(sys.getsizeof(track_uri_to_id) / 1_000_000))

The size of the track_uri -> position mapping dictionary is 20.971608 MB


Let's define the similarity function. Since we are dealing with binary vectors, we can use Jaccard Similarity, since we don't need the information about the single values in the vector.

In [16]:
def jaccard_similarity(vector_1: SparseVector, vector_2: SparseVector) -> float:
  """
  Computes the Jaccard Similarity between two sparse binary vectors
  """
  set1 = set(vector_1.indices)
  set2 = set(vector_2.indices)

  intersection = len(set1.intersection(set2))
  union = len(set1.union(set2))

  similarity = intersection / union

  return similarity

Creating a function that gets in input the playlist to continue, and returns a Dataframe that indicates its similarity with each other playlist in the dataset.

In [17]:
def create_similarity_df(input_vector: DataFrame, rating_vectors_df: DataFrame, similarityFunction: Callable) -> DataFrame:  
  """
  Given a DataFrame with only one row that represents the vector representation of the playlist to continuate, it returns a dataframe containing the similarity between that vector and each
  other playlist vector in the dataset.

  - input_vector: A DataFrame with only one Row, that is the SparseVector representing the input playlist
  - rating_vectors_df: A Dataframe that contains the playlists and their respective vector representation
  """

  input_vector_cached = input_vector.cache()
  input_vector = input_vector.first()[0]
  
  @F.udf(returnType=FloatType())
  def compute_similarity(vector1):
    return jaccard_similarity(vector1, input_vector)

  result_df = rating_vectors_df.withColumn("similarity", compute_similarity(rating_vectors_df.rating_vector))

  input_vector_cached.unpersist()
  
  return result_df

if DEBUG:
  rv_df = songs_embeddings.withColumnRenamed("tracks", "rating_vector")
  # Just to show, we take the first playlist as the playlist to be continued 
  first_playlist_vector = rv_df.limit(1).select("rating_vector").withColumnRenamed("rating_vector","input_vector")
  result_df = create_similarity_df(first_playlist_vector, rv_df, jaccard_similarity)
  result_df.cache()
  result_df.orderBy(F.col("similarity").desc()).show()

If we filter the playlists that have a strictly positive similarity with the input playlist, and order them by descending similarity, we can see that the name (that we assume is very informative for the content of the playlist) is very similar, meaning that the algorithm seems to work!

In [19]:
if DEBUG:
    result_df.filter("similarity > 0").orderBy(F.col("similarity").desc()).show()

Now, in order to suggest some songs to continuate the input playlist, let's take the $k$ top most similar playlists

In [20]:
def get_top_k_results(playlist_pid: int, similarity_df: DataFrame, k: int = 20) -> DataFrame:
  """
  Given the playlist PID and the DataFrames of similarity relative to that playlist, it returns a DataFrame containing the top k most similar playlists.

  - playlist_pid: 
  - similarity_df: 
  """
  return similarity_df.filter((F.col("similarity") > 0) & (F.col("pid") != playlist_pid)).orderBy(F.col("similarity").desc()).limit(k)

if DEBUG:
  first_playlist_pid = rv_df.limit(1).select("pid").first().pid
  top_k_results = get_top_k_results(first_playlist_pid, result_df)
  top_k_results.cache()
  top_k_results.show()

We want to obtain a single embedding for all the $K$ top most similar playlists, that will be the rating vector. We can then pick the indices of the $n$ top greatest values form this vector, and those will be the $n$ songs that we will reccomend.

In order to aggregate the $k$ embeddings into a single one, I decided to take an average, weighted by the similarity value.

In [21]:
def get_input_rating_vector(similarity_df: DataFrame) -> SparseVector:
  """
  Given DataFrames of similarities ordered by similarity in descending order, it returns the vector representation of the first row, which
  is the vector of the input playlist (similarity 1.0)
  """
  return similarity_df.limit(1).select("input_vector").collect()[0].input_vector

def accumulate_top_k_results(top_k_results: DataFrame, input_vector: np.ndarray) -> DataFrame:
  """
  Given a DataFrame that represents the top-k most similar playlists to the input playlist, it returns a
  DataFrame that contains a single row, representing the aggregation of all the playlists' vectors.
  
  The aggregation is just the sum of the vectors, weighted by the relative similarity, and normalized by dividing it by the sum of similarities.
  """

  @F.udf(returnType=VectorUDT())
  def sum_vector(sparse_vectors, similarities):
    similarities = np.array(similarities)
    sparse_vectors = np.array(sparse_vectors)
    acc = np.dot(sparse_vectors.T, similarities) #Compute the sum(vector * similarity) for each vector and similarity
    acc /= similarities.sum() #Normalize the vector
    acc -= (input_vector * acc) #If a song is present in the input playlist, don't consider it
    return SparseVector(acc.size, np.nonzero(acc)[0], acc[np.nonzero(acc)])

  return top_k_results.agg(sum_vector(F.collect_list('rating_vector'), F.collect_list("similarity")).alias('summed'))

if DEBUG:
  t1 = time.time()
  input_vector = first_playlist_vector.first()[0]
  accumulated_vector_df = accumulate_top_k_results(top_k_results, input_vector)
  accumulated_vector_df.cache()
  accumulated_vector_df.show(truncate=False)
  t2 = time.time()
  print(t2-t1)

Now that I have the aggregated vector, I can just take the top-$n$ indices that corresponds to the higher values, and those would be the indices of the songs that are the most relevant to the input playlist.

In [22]:
@F.udf(returnType=ArrayType(
    StructType([
      StructField("pos", IntegerType(), False),
      StructField("confidence", FloatType(), False)
])))
def get_top_n_values(vector: SparseVector, n: int) -> List[Tuple[int, float]]:
  """
  Given the aggregated vector, it returns a list of tuples that map the index of the song to the confidence that that song is relevant for the input playlist.
  The index are only of the top-n most confident results, and are ordered by confidence.
  """
  sorted_elements = vector.toArray().tolist()
  top_n_indices = sorted(range(len(sorted_elements)), key=lambda i: sorted_elements[i], reverse=True)[:n]
  return [(index, sorted_elements[index]) for index in top_n_indices]

if DEBUG:
  t1 = time.time()
  top_n_reccomendations = accumulated_vector_df.withColumn("top_n_recommendations", get_top_n_values(F.col("summed")), F.lit(10)).select(F.explode("top_n_recommendations")).select("col.*")
  top_n_reccomendations.show()
  t2 = time.time()
  print(t2-t1)

Now I can join the indices to the `songs_info_df` DataFrame in order to obtain the `track_uri` of the recommended tracks.

In [23]:
def recommendation_song_info(recommendation: DataFrame, songs_info_df: DataFrame) -> DataFrame:
  return recommendation.join(songs_info_df, "pos")

if DEBUG:
  t1 = time.time()
  songs_info = recommendation_song_info(top_n_reccomendations, song_pos_mapping)
  songs_info.show()
  t2 = time.time()
  print(t2-t1)

### Putting it all togheter
We now define a single function that will get a playlist in input and will reccomend $n$ songs.

In [32]:
def user_based_recommendation(playlist: DataFrame, 
                              mapped_slice_df: DataFrame, 
                              n:int = 50,
                              k: int = 20) -> DataFrame:
  
  """
  Given a DataFrame with a single row that represents the input playlist, generate the DataFrame of n recommendations.
  """
                              
  rv_df = mapped_slice_df.withColumnRenamed("tracks", "rating_vector").cache()
  playlist_vector = map_track_df_to_pos(playlist).select("tracks").withColumnRenamed("tracks", "input_vector").cache()
  similarity_df = create_similarity_df(playlist_vector, rv_df, jaccard_similarity).cache()
  top_k_results = get_top_k_results(playlist.first().pid, similarity_df, k=k).cache()
  input_vector = playlist_vector.select("input_vector").first()[0].toArray()
  accumulated_vector_df = accumulate_top_k_results(top_k_results, input_vector).cache()
  top_n_indices = accumulated_vector_df\
                  .withColumn("top_n_recommendations", get_top_n_values(F.col("summed"), F.lit(n)))\
                  .select(F.explode("top_n_recommendations"))\
                  .select("col.*").cache()

  recommended_songs_info = recommendation_song_info(top_n_indices, song_pos_mapping).cache() 

  playlist_vector.unpersist()
  similarity_df.unpersist()
  top_k_results.unpersist()
  accumulated_vector_df.unpersist()
  top_n_indices.unpersist()
  return recommended_songs_info
  

if DEBUG:
  #Collect and createDataFrame because operations on limit(1) take as long as the entire slice_df, don't know why
  playlist = spark.createDataFrame(slice_df.filter("pid == 1010").limit(1).collect())
  final_recommendation = user_based_recommendation(playlist, songs_embeddings, jaccard_similarity, n=5)

In [26]:
if DEBUG:
    final_recommendation.show()

## Performance Evaluation

Since now we operated on the entire dataset, but for performance evaluation we will generate some recommendations for each playlist in the training set, and then compare the recommended songs with the songs in the test set.

In [27]:
TRAIN_DF_PATH = os.path.join(SAVED_DFS_PATH, f"train_df-{NUM_PLAYLISTS}.json")
TEST_DF_PATH = os.path.join(SAVED_DFS_PATH, f"test_df-{NUM_PLAYLISTS}.json")
train_df = spark.read.schema(playlist_schema).json(TRAIN_DF_PATH)
test_df = spark.read.schema(playlist_schema).json(TEST_DF_PATH)

In [28]:
train_df.show()
test_df.show()

+-------------+-------------+----+-----------+----------+----------+-------------+--------------------+---------+-----------+-----------+
|         name|collaborative| pid|modified_at|num_tracks|num_albums|num_followers|              tracks|num_edits|duration_ms|num_artists|
+-------------+-------------+----+-----------+----------+----------+-------------+--------------------+---------+-----------+-----------+
|       disney|        false|1000| 1457827200|       189|        16|            1|[{31, Daughters o...|        4|   31428282|         65|
|Indie Electro|        false|1001| 1417824000|       165|        18|            2|[{117, Boards of ...|        2|   38241566|          8|
|  jack & jack|        false|1002| 1465430400|        17|        14|            1|[{14, Jack & Jack...|        3|    3549358|          3|
|        vibes|        false|1003| 1498435200|       225|       195|            2|[{119, PREP, spot...|       91|   51242585|        157|
|        Indie|        false|1004|

In [29]:
def precision_at_k(recommendations: DataFrame, ground_truth: DataFrame, num_of_recommendations: int) -> float:
    """
    Calculates precision at k for the recommendations.
    """
    recommended_relevant_tracks = recommendations.join(ground_truth, "track_uri").cache()
    reccomended_relevant_tracks_count = recommended_relevant_tracks.count() #this can be top_n_results.join in order to be more performant
    recommended_relevant_tracks.unpersist()
    precision = reccomended_relevant_tracks_count / float(num_of_recommendations)

    return precision

def normalized_discounted_cumulative_gain(recommendations: DataFrame, ground_truth: DataFrame, num_of_recommendations: int) -> float:
  """
  Calculates the Normalized Discounted Cumulative Gain between the DataFrame of recommendations and the DataFrame of ground truth.
  """
  recommendations = recommendations.orderBy(F.col("confidence").desc())
  recommendations_list = recommendations.collect()
  cumulative_gain = 0

  intersection = recommendations.join(ground_truth, "track_uri").count()
  if intersection == 0: return 0

  ideal_cumulative_gain = 1 + np.array([(1 / math.log(i, 2)) for i in range(2, 2+intersection)]).sum()
  for index, row in enumerate(recommendations_list):
    i = index + 1
    is_rel = ground_truth.filter(F.col("track_uri").isin(row.track_uri)).count() > 0
    rel = 1 if is_rel else 0
    if i == 1:
      cumulative_gain += rel
    else:
      cumulative_gain += (rel / math.log(i, 2))
  return cumulative_gain / ideal_cumulative_gain

Let's define the function that will perform the evaluation pipeline for a single playlist.

In [30]:
def evaluate(pid: int) -> Tuple[DataFrame, float]:
    """
    Perform the evaluation for a given playlist.
    """
    playlist_train = train_df.filter(f"pid == {pid}").cache()
    playlist_test = test_df.filter(f"pid == {pid}").cache()
    ground_truth = playlist_test.select(F.explode("tracks")).select("col.*").cache()
    num_of_recommendations = ground_truth.count()
    recommendations = user_based_recommendation(playlist_train, 
                                                songs_embeddings, 
                                                n=num_of_recommendations,
                                                k = 10).cache()
    precision = precision_at_k(recommendations, ground_truth, num_of_recommendations)
    gain = normalized_discounted_cumulative_gain(recommendations, ground_truth, num_of_recommendations)

    playlist_train.unpersist()
    playlist_test.unpersist()
    ground_truth.unpersist()
    return playlist_train, playlist_test, ground_truth, recommendations, precision, gain

Now we can finally perform the evaluation on a set of playlists. I decided to evaluate the algorithm on 1000 playlists. Since each evaluation step takes about 30 seconds (30,000 seconds are about 8 hours), I used checkpointing in order to interrupt the evaluation and restart it from the same point.

In [33]:
LAST_CHECKPOINT_INDEX = 0
EVALUATION_RESULTS_PATH = os.path.join(GDRIVE_DATA_DIR, f"user_base_evaluation", "UB_evaluation_results_FINAL")
def perform_evaluation():
  SAMPLING_FRACTION = 0.01
  sampled_playlists = train_df.sample(False, SAMPLING_FRACTION, seed=42).cache()

  results = []
  for index, row in enumerate(tqdm(sampled_playlists.collect(), desc="Performing evaluation")):
      if index <= LAST_CHECKPOINT_INDEX: continue 
      
      CHECKPOINT_RESULTS = os.path.join(GDRIVE_DATA_DIR, "user_base_evaluation", f"UB_evaluation_results_check_{index}")
      pid = row['pid']
      train, test, gt, rec, prec, gain = evaluate(pid)
      results.append((prec, gain))
      print((prec, gain))
      if index % 10 == 0:
         with open(CHECKPOINT_RESULTS, "w") as f:
            json.dump(results, f)
  with open(EVALUATION_RESULTS_PATH, "w") as f:
    json.dump(results, f)
  
  sampled_playlists.unpersist()
  return results


results = perform_evaluation()

23/06/29 18:11:07 WARN CacheManager: Asked to cache already cached data.


Performing evaluation:   0%|          | 0/1024 [00:00<?, ?it/s]

23/06/29 18:11:07 WARN CacheManager: Asked to cache already cached data.
23/06/29 18:11:07 WARN CacheManager: Asked to cache already cached data.
23/06/29 18:11:07 WARN CacheManager: Asked to cache already cached data.
23/06/29 18:11:07 WARN CacheManager: Asked to cache already cached data.
23/06/29 18:11:08 WARN CacheManager: Asked to cache already cached data.
23/06/29 18:11:08 WARN CacheManager: Asked to cache already cached data.        
23/06/29 18:11:08 WARN CacheManager: Asked to cache already cached data.
23/06/29 18:11:09 WARN CacheManager: Asked to cache already cached data.


(0.07692307692307693, 0.5)


23/06/29 18:11:29 WARN CacheManager: Asked to cache already cached data.
23/06/29 18:11:30 WARN CacheManager: Asked to cache already cached data.
23/06/29 18:11:31 WARN CacheManager: Asked to cache already cached data.


(0.0, 0)


23/06/29 18:11:47 WARN CacheManager: Asked to cache already cached data.
23/06/29 18:11:47 WARN CacheManager: Asked to cache already cached data.
23/06/29 18:11:48 WARN CacheManager: Asked to cache already cached data.
ERROR:root:KeyboardInterrupt while sending command.                 (0 + 8) / 8]
Traceback (most recent call last):
  File "/Users/dov/miniconda3/envs/dl/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/dov/miniconda3/envs/dl/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/dov/miniconda3/envs/dl/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

Let's load the `results`, average them and see how the model performed.

In [ ]:
LAST_CHECKPOINT_RESULTS = os.path.join("./data", "user_base_evaluation", f"evaluation_results_check_500")
with open(LAST_CHECKPOINT_RESULTS, "r") as f:
    results = json.load(f)

avg_prec = np.array(results).mean()
avg_prec, avg_gain = 0, 0
for prec, gain in results:
  avg_prec += prec
  avg_gain += gain 
tot = len(results)
avg_prec /= tot
avg_gain /= tot
avg_prec, avg_gain

(0.10053558551616848, 0.28256513840537667)

We can see that the model has a precision of about 0.10 and a NDCG of 0.28, which isn't bad taking into account that state of the art models with 1M playlists for train have 0.2 precision and 0.38 NDCG.

# Fighting against the curse of dimensionality: Matrix Factorization

We want to define $\mathbf{x}_u \in \mathbb{R}^d$ $d$-dimensional vector that represents the user $u$, and $\mathbf{w}_i \in \mathbb{R}^d$ vector that represent the item $i$.

We then can estimate the rating of user $u$ for the item $i$ by computing
\begin{equation}
\hat{r}_{u, i}=\mathbf{x}_u^T \cdot \mathbf{w}_i=\sum_{j=1}^d x_{u, j} w_{j, i}
\end{equation}
Or, in matrix notation,

\begin{equation}
\underbrace{R}_{m \times n} =
\underbrace{X}_{m \times d}
\underbrace{W^T}_{d \times n}
\end{equation}

### How to learn $X$ and $W$
The matrix $R$ is partially known and filled with the observations inside the dataset $\mathcal{D}$. In order to learn the latent factor representations $X$ and $W$, we minimize the following loss function:
\begin{equation}
L(X, W)=\sum_{(u, i) \in \mathcal{D}}\underbrace{\left(r_{u, i}-\mathbf{x}_u^T \cdot \mathbf{w}_i\right)^2}_{\text{squared error term}}+\underbrace{\lambda\left(\sum_{u \in \mathcal{D}}\left\|\mathbf{x}_u\right\|^2+\sum_{i \in \mathcal{D}}\left\|\mathbf{w}_i\right\|^2\right)}_{\text{regularization term}}
\end{equation}

We can then minimize the loss using Stochastic Gradient Descent or Alternating Least Squares.

# Matrix Factorization
Generate a matrix Y where each column represent a playlist and each row represent a song, the (i,j) entry will be 1 if the playlist contains the song, 0 otherwise.

In [ ]:
# Throw error in order to not execute the following code
raise ValueError()

In [ ]:
import pyspark.sql.functions as f
from pyspark.sql.functions import explode
spark.conf.set("spark.sql.pivotMaxValues", 1000000)

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.functions import expr


In [ ]:
from pyspark.sql.functions import explode
import random
tracks_df = slice_df.select("pid", explode("tracks").alias("track")).select("pid", "track.track_uri")
tracks_df = tracks_df.withColumn("rating", F.lit(1))
# tracks_df = tracks_df.withColumn("rating", (rand() * 10 + 1).cast("integer"))

In [ ]:
tracks_df.show()

In [ ]:
# # Explode the tracks array column into multiple rows
# # tracks_df = slice_df.select("pid", explode("tracks").alias("track"))
# # tracks_df = slice_df.select("pid", "tracks", "tracks")
# tracks_df = slice_df.select("pid", explode("tracks").alias("track")).select("pid", "track.track_uri", "track.pos")

# # Select relevant columns and add a rating column with value 1
# playlist_track_df = tracks_df.withColumn("rating", lit(1))

# # Get distinct track_uri values and join with playlist_track_df
# all_tracks_df = slice_df.select(explode("tracks").alias("track")).select("track.track_uri").distinct()
# all_playlists_df = slice_df.select("pid").distinct()

# all_against_all = all_tracks_df.join(all_playlists_df).distinct()

# from pyspark.sql.functions import when, col

# # playlist_track_rating_df = playlist_track_df.join(all_against_all, ["pid", "track_uri"], "left_outer") \
# #     .withColumn("rating", when(col("pos").isNull(), 0).otherwise(1))

# playlist_track_rating_df = all_against_all.join(playlist_track_df, ["pid", "track_uri"], "left_outer") \
#     .withColumn("rating", when(col("pos").isNull(), 0).otherwise(1)) \
#     .drop("pos")


In [ ]:
playlist_track_rating_df = tracks_df.withColumn("song_id", dense_rank().over(Window.orderBy("track_uri")))

In [ ]:
playlist_track_rating_df.show(truncate=False)

In [ ]:
als = ALS(userCol="pid", itemCol="song_id", ratingCol="rating", nonnegative=True, coldStartStrategy="drop")

In [ ]:
from typing import Tuple
import random

def train_test_split(df: DataFrame, split_ratio: float, seed: Optional[int] = None) -> Tuple[DataFrame, DataFrame]:
  random.seed(seed)
  distinct_pids = df.select("pid").distinct().rdd.map(lambda x: x[0]).collect()
  random.shuffle(distinct_pids)
  split_index = int(len(distinct_pids) * split_ratio)
  train_pids = distinct_pids[:split_index]
  test_pids = distinct_pids[split_index:]
  train_df = df.filter(col("pid").isin(train_pids))
  test_df = df.filter(col("pid").isin(test_pids))
  return train_df, test_df



In [ ]:
training, test = playlist_track_rating_df.randomSplit([0.8, 0.2], seed=42)

In [ ]:
model = als.fit(training)

In [ ]:
predictions = model.transform(test)

In [ ]:
predictions.show()

In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [ ]:
predictions.filter(col("prediction") != "NaN").count(), predictions.filter(col("prediction") == "NaN").count()

In [ ]:
rmse

In [ ]:
subset = playlist_track_rating_df.select("pid").distinct().limit(1)
subUserRecs = model.recommendForUserSubset(subset, 10)

In [ ]:
subset.show()

In [ ]:
subUserRecs.show(truncate=False)

In [ ]:
def song_name_from_id(song_id: int, reverse_lookup: DataFrame) -> str:
  return 
  
def interpretRecommendation(recommended_result: DataFrame) -> str:
  return

In [ ]:
userRecs = model.recommendForAllUsers(1).orderBy("recommendations")
userRecs.show(truncate=False)
userRecs.count()

In [ ]:
slice_df.filter(col("pid") == 1710).select(explode("tracks.track_name")).show()

In [ ]:
track_uris = playlist_track_rating_df.filter(col("song_id") == 588).select("track_uri")
track_uris.first()